# Naive Bayes (the easy way)

We'll cheat by using sklearn.naive_bayes to train a spam classifier! Most of the code is just loading our training data into a pandas DataFrame that we can play with:

In [19]:
import os
import io
import numpy
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

def readFiles(path):
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            path = os.path.join(root, filename)

            inBody = False
            lines = []
            f = io.open(path, 'r', encoding='latin1')
            for line in f:
                if inBody:
                    lines.append(line)
                elif line == '\n':
                    inBody = True
            f.close()
            message = '\n'.join(lines)
            yield path, message


def dataFrameFromDirectory(path, classification):
    rows = []
    index = []
    for filename, message in readFiles(path):
        rows.append({'message': message, 'class': classification})
        index.append(filename)

    return DataFrame(rows, index=index)

data = DataFrame({'message': [], 'class': []})
#C:\Users\Vijay\Desktop\OPJ Workshop\Day3\emails\ham
data = data.append(dataFrameFromDirectory('C:\\Users\\Vijay\\Desktop\\OPJ Workshop\\Day3\\emails\\spam', 'spam'))

#data = data.append(dataFrameFromDirectory('C:/Users/Vijay/Desktop/workshop flow/Day 2/Day2_IPYNB_Files/emails/spam', 'spam'))
#data = data.append(dataFrameFromDirectory('C:/Users/Vijay/Desktop/workshop flow/Day 2/Day2_IPYNB_Files/emails/ham', 'ham'))
data = data.append(dataFrameFromDirectory('C:\\Users\\Vijay\\Desktop\\OPJ Workshop\\Day3\\emails\\ham', 'ham'))

length= len(data)
length

3000

Let's have a look at that DataFrame:

In [11]:
data.tail(30)

,class,message
C:\Users\Vijay\Desktop\OPJ Workshop\Day3\emails\ham\02471.18281d43dc0775e915267c2ea5170f1f,ham,"This is possible, however using SA as a block ..."
C:\Users\Vijay\Desktop\OPJ Workshop\Day3\emails\ham\02472.5c879dd55c3d4171e1787e8529bbd7e1,ham,\n\n--- Martin Adamson <martin@srv0.ems.ed.ac....
C:\Users\Vijay\Desktop\OPJ Workshop\Day3\emails\ham\02473.207afa13ad7d745dfd1344f84531ac16,ham,"----- Original Message -----\n\nFrom: ""Tim Cha..."
C:\Users\Vijay\Desktop\OPJ Workshop\Day3\emails\ham\02474.c76ffef81a2529389e6c3bbb172184d7,ham,"\n\n> Mr Tim Chapman, freelance gentleman of l..."
C:\Users\Vijay\Desktop\OPJ Workshop\Day3\emails\ham\02475.9277ee243e3f51fa53ed6be55798d360,ham,"Smith, Graham - Computing Technician wrote:\n\..."
C:\Users\Vijay\Desktop\OPJ Workshop\Day3\emails\ham\02476.de1d459426662492dd1235046b504c3d,ham,Geege wrote a strange story:\n\n>I know a guy ...
C:\Users\Vijay\Desktop\OPJ Workshop\Day3\emails\ham\02477.07b2069e9827cfd6f97d07eea2913d57,ham,\n\n[Paul Moore]\n\n> but let's walk before...
C:\Users\Vijay\Desktop\OPJ Workshop\Day3\emails\ham\02478.40723f38488bddaf5a24ef2a91679c75,ham,"On Mon, Nov 25, 2002 at 06:54:49PM +0000, Phil..."
C:\Users\Vijay\Desktop\OPJ Workshop\Day3\emails\ham\02479.14365bcad3a60fcf24c5c1813f6291fb,ham,\n\nI don't know how one can expect better and...
C:\Users\Vijay\Desktop\OPJ Workshop\Day3\emails\ham\02480.72714df60c9be29d6f7985c777cbfc13,ham,"No, you need to learn how declarations work in..."


Now we will use a CountVectorizer to split up each message into its list of words, and throw that into a MultinomialNB classifier. Call fit() and we've got a trained spam filter ready to go! It's just that easy.

In [13]:
vectorizer = CountVectorizer()
counts = vectorizer.fit_transform(data['message'].values)

classifier = MultinomialNB()
targets = data['class'].values
classifier.fit(counts, targets)


array(['spam', 'spam', 'spam', ..., 'ham', 'ham', 'ham'], dtype=object)

Let's try it out:

In [15]:
examples = ['I had the same problem when installing', "Hi Bob, how about a game of golf tomorrow?"]
example_counts = vectorizer.transform(examples)
predictions = classifier.predict(example_counts)
predictions

array(['ham', 'ham'], dtype='<U4')

## Activity

Our data set is small, so our spam classifier isn't actually very good. Try running some different test emails through it and see if you get the results you expect.

If you really want to challenge yourself, try applying train/test to this spam classifier - see how well it can predict some subset of the ham and spam emails.